# **1. SET-UP**

In [1]:
from google.colab import drive
drive.mount('/content/sample_data/MyDrive')

Mounted at /content/sample_data/MyDrive


In [2]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow import keras

# **2. DATA LOADING**

In [3]:
data_dir = ('/content/sample_data/MyDrive/MyDrive/lane_classifier/data')
dataset = tf.keras.utils.image_dataset_from_directory(data_dir, label_mode='categorical',batch_size = 64)

Found 2287 files belonging to 3 classes.


In [4]:
data_iterator = dataset.as_numpy_iterator()
batch = data_iterator.next()

In [5]:
len(batch)

2

In [6]:
import tensorflow as tf

def resize_function(x, y):
    resized_x = tf.image.resize(x, (224, 224))

    return resized_x, y

# Apply the resize function to each element in the dataset
dataset = dataset.map(resize_function)

dataset = dataset.map(lambda x,y : (x/255,y))

In [7]:
scaled_iterator = dataset.as_numpy_iterator()
scaled_batch = scaled_iterator.next()
scaled_batch[0].max()

1.0

In [8]:
train_size = int(len(dataset)*.7)+1
val_size = int(len(dataset)*.1)+2
test_size = int(len(dataset)*.1)+2

In [9]:
train_data = dataset.take(train_size)
val_data = dataset.skip(train_size).take(val_size)
test_data = dataset.skip(train_size+val_size).take(test_size)

In [10]:
def extract_features_labels(image, label):
    return image, label

x_train, y_train = zip(*train_data.map(extract_features_labels))
x_val, y_val = zip(*val_data.map(extract_features_labels))
x_test, y_test = zip(*test_data.map(extract_features_labels))

In [11]:
x_train = tf.concat(x_train, axis=0).numpy()
y_train = tf.concat(y_train, axis=0).numpy()
x_val = tf.concat(x_val, axis=0).numpy()
y_val = tf.concat(y_val, axis=0).numpy()
x_test = tf.concat(x_test, axis=0).numpy()
y_test = tf.concat(y_test, axis=0).numpy()

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


# **3. MODEL ARCHITECTURE**

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, BatchNormalization

In [14]:
# original
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (224,224,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense( 256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense( 3, activation = 'softmax'))


In [16]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 16)        0

# LR = 0.0001

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

def lr_scheduler(epoch):
    return 0.0001 * (0.5 ** (epoch // 20))
reduce_lr = LearningRateScheduler(lr_scheduler)

mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', save_best_only=True, mode='max')


In [17]:
# lr = 0.0001
EPOCHS = 30
BS = 64

history = model.fit(
    aug.flow(x_train,y_train, batch_size=BS),
    validation_data=(x_val,y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS,
    callbacks=[reduce_lr,mc])

Epoch 1/30
26/26 [==============================] - 26s 724ms/step - loss: 0.9712 - accuracy: 0.5643 - val_loss: 0.7575 - val_accuracy: 0.7531 - lr: 1.0000e-04
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 20s 770ms/step - loss: 0.6461 - accuracy: 0.7800 - val_loss: 0.4527 - val_accuracy: 0.8844 - lr: 1.0000e-04
Epoch 3/30
26/26 [==============================] - 20s 756ms/step - loss: 0.4094 - accuracy: 0.8894 - val_loss: 0.3052 - val_accuracy: 0.9594 - lr: 1.0000e-04
Epoch 4/30
26/26 [==============================] - 19s 748ms/step - loss: 0.2314 - accuracy: 0.9519 - val_loss: 0.2125 - val_accuracy: 0.9656 - lr: 1.0000e-04
Epoch 5/30
26/26 [==============================] - 21s 779ms/step - loss: 0.1386 - accuracy: 0.9669 - val_loss: 0.1702 - val_accuracy: 0.9438 - lr: 1.0000e-04
Epoch 6/30
26/26 [==============================] - 19s 745ms/step - loss: 0.1006 - accuracy: 0.9724 - val_loss: 0.1464 - val_accuracy: 0.9688 - lr: 1.0000e-04
Epoch 7/30
26/26 [==============================] - 21s 798ms/step - loss: 0.0809 - accuracy: 0.9814 - val_loss: 0.1156 - val_accuracy: 0.9719 - lr: 1.0000e-04
Epoch 8/30
26/26 [==============================] -

# LR = 0.01

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

def lr_scheduler(epoch):
    return 0.0001 * (0.5 ** (epoch // 20))
reduce_lr = LearningRateScheduler(lr_scheduler)

mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', save_best_only=True, mode='max')


In [19]:
# lr = 0.01
EPOCHS = 30
BS = 64

history = model.fit(
    aug.flow(x_train,y_train, batch_size=BS),
    validation_data=(x_val,y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS,
    callbacks=[reduce_lr,mc])

Epoch 1/30
26/26 [==============================] - 20s 766ms/step - loss: 23.3698 - accuracy: 0.4645 - val_loss: 1.0752 - val_accuracy: 0.4469 - lr: 0.0100
Epoch 2/30
26/26 [==============================] - 20s 772ms/step - loss: 1.0655 - accuracy: 0.4615 - val_loss: 1.0732 - val_accuracy: 0.4469 - lr: 0.0100
Epoch 3/30
26/26 [==============================] - 19s 747ms/step - loss: 1.0594 - accuracy: 0.4615 - val_loss: 1.0751 - val_accuracy: 0.4469 - lr: 0.0100
Epoch 4/30
26/26 [==============================] - 20s 757ms/step - loss: 1.0612 - accuracy: 0.4615 - val_loss: 1.0724 - val_accuracy: 0.4469 - lr: 0.0100
Epoch 5/30
26/26 [==============================] - 19s 739ms/step - loss: 1.0598 - accuracy: 0.4615 - val_loss: 1.0747 - val_accuracy: 0.4469 - lr: 0.0100
Epoch 6/30
26/26 [==============================] - 20s 754ms/step - loss: 1.0585 - accuracy: 0.4615 - val_loss: 1.0758 - val_accuracy: 0.4469 - lr: 0.0100
Epoch 7/30
26/26 [==============================] - 20s 743ms/s